In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-126762")
exp = Experiment(workspace = ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126762
Azure region: southcentralus
Subscription id: 55e71b9d-a209-42c0-8818-ca9cc885909c
Resource group: aml-quickstarts-126762


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4

def cluster_validation():
    cluster = 'my-cluster'
    try:
        compute_cluster = ComputeTarget(workspace = ws, name = cluster)
        print('Found existing compute cluster!')
    except ComputeTargetException:
        config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2',max_nodes = 4)
        compute_cluster = ComputeTarget.create(ws, cluster, config)
    compute_cluster.wait_for_completion(show_output = True)
    return compute_cluster

#### a) Creating cluster for the first time

In [3]:
compute_cluster = cluster_validation()

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


#### b) Calling the function to create cluster again

In [4]:
compute_cluster = cluster_validation()

Found existing compute cluster!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,randint,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        "--max_iter" : choice(5,10,20,40),
        "--C": uniform(0.5,1.0)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1,delay_evaluation = 1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",entry_script = 'train.py',compute_target = compute_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                    hyperparameter_sampling = ps,
                                    policy = policy,
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 25,
                                    max_concurrent_runs = 4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdconfig = exp.submit(config = hyperdrive_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [7]:
RunDetails(hdconfig).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
assert(hdconfig.get_status() == "Completed")

In [10]:
import joblib
# Get your best run and save the model from that run.

best_run = hdconfig.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_da1813a9-c46b-4696-8976-b9755a019132_7

 Accuracy: 0.9074355083459787


In [11]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.668893350608258', '--max_iter', '40']


#### Listing the model files uploaded during the run

In [12]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_ba8120d2b6f6fe59650536af62618a3a7921ddc061fcee49fefaf14f82121641_d.txt',
 'azureml-logs/65_job_prep-tvmps_ba8120d2b6f6fe59650536af62618a3a7921ddc061fcee49fefaf14f82121641_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_ba8120d2b6f6fe59650536af62618a3a7921ddc061fcee49fefaf14f82121641_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/102_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log']

#### Saving the model under the workspace for deployment

In [13]:
model = best_run.register_model(model_name = 'best_model', model_path = './')

In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path = url)

In [15]:
from train import clean_data

x, y = clean_data(ds)

In [16]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = 'classification',
    primary_metric = 'accuracy',
    training_data = ds,
    label_column_name = 'y',
    n_cross_validations = 7,
    compute_target = compute_cluster,
    iterations = 5)

In [17]:
# Submit your automl run

experiment = Experiment(ws, "automl_exp")
run = experiment.submit(config = automl_config, show_output = True)

Running on remote.
Running on remote compute: my-cluster
Parent Run ID: AutoML_7e62a313-77dd-4e54-be40-a453d45c60ce

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+-------------

In [18]:
# Retrieve and save your best automl model.

best_run, best_model = run.get_output()
best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-126762', subscription_id='55e71b9d-a209-42c0-8818-ca9cc885909c', resource_group='aml-quickstarts-126762'), name=automl_best_model.pkl, id=automl_best_model.pkl:1, version=1, tags={}, properties={})

#### Deleting the cluster before termination

In [19]:
compute_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

